In [1]:
from src.recommender import *

In [2]:
recommender = Recommender()

In [3]:
recommender.get_id("Wolf's Rain")

Found title: Wolf's Rain


In [4]:
recommender.get_id("Ergo Proxy")

Found title: Ergo Proxy


In [5]:
recommender.get_id("Texhnolyze")

Found title: Texhnolyze


In [7]:
recommender._calc_similarities()

In [9]:
recommender.get_recommendations()

Neon Genesis Evangelion
Casshern Sins
Land of the Lustrous
Neon Genesis Evangelion: The End of Evangelion
Cowboy Bebop: The Movie - Knockin' on Heaven's Door
Wolf's Rain OVA
Ghost in the Shell: Stand Alone Complex 2nd GIG
PSYCHO-PASS
Ghost in the Shell
Akira
